---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="4dd4fba5" -->
# Table des matières
1. [Le traitement des données dupliquées](#le-traitement-des-données-dupliquées)
1. [Le traitement des données aberrantes](#le-traitement-des-données-aberrantes)
  1. [Génération des données](#génération-des-données)
  1. [Élimination des données aberrantes](#élimination-des-données-aberrantes)
    1. [Covariance robuste](#covariance-robuste)
    1. [SVM monoclasse](#svm-monoclasse)
    1. [Forêt d'isolation](#forêt-disolation)
    1. [Facteur d'aberration local](#facteur-daberration-local)
    1. [Test de Tukey pour les valeurs extrêmes](#test-de-tukey-pour-les-valeurs-extrêmes)
    1. [Estimation de densité par noyau (KDE)](#estimation-de-densité-par-noyau-kde)
1. [Conclusion](#conclusion)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import time
import warnings

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import make_blobs, make_moons

sns.set(color_codes=True)
warnings.filterwarnings("ignore")
matplotlib.rcParams["contour.negative_linestyle"] = "solid"


seed = 42
np.random.seed(seed)



<!-- #region colab_type="text" id="1585634b" -->
Dans ce module, nous allons nous concentrer sur les deux premières étapes du prétraitement des données. La
première concerne les données dupliquées. La seconde porte sur la détection et l'élimination
des valeurs aberrantes dans les jeux de données.

La présence de données dupliquées n'affecte pas vraiment l'entraînement d'un modèle. Toutefois, cela a des conséquences sur ses performances et l'interprétation des résultats. La solution est simple : éliminer ces valeurs.

Le problème des valeurs aberrantes est plus sérieux. Il y a plusieurs raisons pour lesquelles on peut observer de
telles valeurs dans des résultats expérimentaux. En voici quelques-unes:


- erreurs instrumentales,
- bruit naturel dans les données,
- mauvaise entrée dans un fichier Excel,
- valeur insérée intentionnellement afin de tester un algorithme,
- nouveau phénomène!


Le dernier point est le plus important. Il se peut que la donnée soit correcte, mais totalement
inattendue. Avec un peu de chance, une investigation plus poussée peut lever le voile sur un nouveau phénomène scientifique! Il
faut garder cette éventualité à l'esprit avant d'éliminer toute valeur aberrante. C'est un bel
exemple de l'expression « Jeter le bébé avec l'eau du bain ». Le bébé est le point inattendu; l'eau
du bain est le reste des données aberrantes.
<!-- #endregion -->

<!-- #region id="a2718f78" -->
# <a id=le-traitement-des-données-dupliquées>Le traitement des données dupliquées</a>
<!-- #endregion -->

<!-- #region id="aa42a1e5" -->
Cette première section se concentre sur quelques opérations de base en nettoyage de données. Dans l'exemple suivant,
les données d'un  jeu de données ont été enregistrées dans un [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) (`df`) de la librairie Pandas. L'élimination des données dupliquées (ou doublons) est une étape nécessaire qu'il faut effectuer bien avant celle de la détection des valeurs aberrantes.

Lors de la collecte des données, il arrive souvent que l'on doive fusionner plusieurs ensembles de données entre eux.
Cela risque d'entraîner la duplication de plusieurs valeurs. 

Ces données dupliquées peuvent fausser l'importance de certaines caractéristiques/variables dans l'ensemble de données. 
Si une caractéristique est systématiquement associée à des instances dupliquées, le modèle d'apprentissage automatique peut lui attribuer une importance supérieure à celle qu'elle mérite, ce qui entraîne une sélection ou une interprétation incorrecte de la caractéristique.

De plus, les données dupliquées peuvent se retrouver dans les ensembles d'entraînement **et** de test. 
Cela augmente alors artificiellement les performances des modèles en test.

<!-- #endregion -->

<!-- #region id="3c8e28ca" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/alice.jpeg"  width="400" />
    <div>
    <font size="0.5">Image Source: https://boingboing.net/2005/10/20/disney-launches-alic.html</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="c29f097c" -->
Les opérations suivantes sont souvent effectuées lors de cette étape de débroussaillage.

**Compter les valeurs uniques dans la colonne `Nom`:**



In [ ]:
df.groupby('Nom').id_user.nunique()



**Montrer les valeurs dupliquées dans la colonne `Nom`:**



In [ ]:
pd.concat(i for _, i in df.groupby('Nom') if len(i) > 1)



**Montrer les valeurs dupliquées dans plus d'une colonne:**



In [ ]:
pd.concat(i for _, i in df.groupby(['Nom1', 'Nom2', 'Nom3']) if len(i) > 1)




**Retirer les données dupliquées dans la colonne `Nom`:**



In [ ]:
df = df.drop_duplicates(subset='Nom', keep=False)



Les doublons sont parfois faciles à repérer visuellement dans une colonne après quelques étapes de prétraitement. Le simple affichage des données d'une colonne en ordre croissant peut révéler des anomalies. Par exemple, si la colonne contient des valeurs entre 0 et 1 000 et que vous notez sept valeurs consécutives de 102,37 alors ça vaudrait la peine d'investiguer. La plupart du temps, des exemples "différents" deviennent des doublons après prétraitement.

> À noter que ces quelques commandes très utiles ont été extraites de l'aide-mémoire en prétraitement des données d'[Elisabeth Reitmayr](https://github.com/lis365b/data_analysis/blob/master/data_prep_python_cheatsheet.md).
Jetez un coup d'oeil pour y apprendre plein de trucs utiles.

<!-- #endregion -->

<!-- #region id="171a9c94" -->
# <a id=le-traitement-des-données-aberrantes>Le traitement des données aberrantes</a>
<!-- #endregion -->

<!-- #region id="413f6ae5" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/fish-outlier.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://towardsdatascience.com/a-brief-overview-of-outlier-detection-techniques-1e0b2c19e561</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region colab_type="text" id="366f6882" -->
La séquence suivante est inspirée des [exemples de code](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html)
de la librairie Scikit-learn.

Nous allons commencer par générer plusieurs ensembles de données en y incluant des données aberrantes, c'est-à-dire,
hors de la distribution des données d'intérêt. Puis, nous allons utiliser diverses techniques
permettant (avec plus ou moins de succès) de déterminer quelles données font partie de la distribution
initiale, et lesquelles n'en font pas partie.

N'hésitez pas à jouer avec les paramètres des différents algorithmes pour voir l'implication de chacun!

<!-- #endregion -->

<!-- #region id="508fb9ab" -->
## <a id=génération-des-données>Génération des données</a>
<!-- #endregion -->

<!-- #region id="b25f8974" -->
Nous allons générer cinq ensembles de données 2-D avec des distributions spatiales de complexités croissantes
et y inclure des données aberrantes. Nous allons ensuite examiner diverses techniques permettant
de déterminer quelles données font partie de la distribution initiale,
et lesquelles n'en font pas partie.

N'hésitez pas à jouer avec les paramètres des différents algorithmes pour voir l'implication de chacun!

> À noter que dans la littérature anglophone, les bonnes données sont des *inliers* et les valeurs aberrantes
sont des *outliers*. Pensez à une file de gens alignés. Ceux qui sont dans la ligne (*inliers*) sont
correctement positionnés. Ceux qui en sortent (*outliers*) sont des gens qui attirent l'attention
(*outstanding people*). Voilà l'origine de ces termes.
<!-- #endregion -->



In [ ]:
# Paramètres des exemples
n_samples = 300

# Quantité de données aberrantes (en %)
outliers_fraction = 0.25


In [ ]:
# Définition des jeux de données avec données aberrantes selon les paramètres précédents

n_outliers = int(outliers_fraction * n_samples)
n_inliers = n_samples - n_outliers

blobs_params = dict(random_state=0, n_samples=n_inliers, n_features=2)
# Cinq datasets:
datasets = [
    # Un seul nuage de point (blob en anglais)
    make_blobs(centers=[[0, 0], [0, 0]], cluster_std=0.5, **blobs_params)[0],
    # Deux nuages de points séparés
    make_blobs(centers=[[2, 2], [-2, -2]], cluster_std=[0.5, 0.5], **blobs_params)[0],
    # Deux nuages de points séparés dont un plus étalé
    make_blobs(centers=[[2, 2], [-2, -2]], cluster_std=[1.5, 0.3], **blobs_params)[0],
    # Deux croissants de lune
    4.0
    * (
        make_moons(n_samples=n_samples, noise=0.05, random_state=0)[0]
        - np.array([0.5, 0.25])
    ),
    # Distribution aléatoire uniforme
    14.0 * (np.random.RandomState(seed).rand(n_samples, 2) - 0.5),
]

# Grillage (mesh en anglais) pour afficher les contours de décision lorsque
# disponibles.
xx, yy = np.meshgrid(np.linspace(-7, 7, 150), np.linspace(-7, 7, 150))



<!-- #region id="05cc04df" -->
L'affichage montre les bonnes données, les valeurs aberrantes et les contours de décision générés par certains algorithmes.
<!-- #endregion -->



In [ ]:
def plot_datasets(name="", algorithm=None, addOutliers=True):
    # Taille des figures
    plt.figure(figsize=(len(datasets) * 2 + 3, 3))
    plt.subplots_adjust(
        left=0.02, right=0.98, bottom=0.001, top=0.96, wspace=0.05, hspace=0.01
    )

    plot_num = 1
    # Garantit la reproductibilité
    rng = np.random.RandomState(42)

    for i_dataset, X in enumerate(datasets):

        y_pred = np.repeat(0, len(X))
        # Ajout des valeurs aberrantes
        if addOutliers:
            X = np.concatenate(
                [X, rng.uniform(low=-6, high=6, size=(n_outliers, 2))], axis=0
            )

            # On met la vérité dans la prédiction si on ne fait pas
            # de prédiction.
            y_pred = np.concatenate([y_pred, np.repeat(1, n_outliers)], axis=0)

        t0 = 0
        t1 = 0
        ax = plt.subplot(1, len(datasets), plot_num)

        # Si on veut voir la prédiction dans l'affichage :
        if algorithm is not None:
            t0 = time.time()
            algorithm.fit(X)
            t1 = time.time()
            if i_dataset == 0:
                plt.title(name, size=18)

            # Traite les données et prédit si chaque point est une valeur
            # aberrante ou non
            if name == "Local Outlier Factor":
                # LOF n'implémente pas la méthode predict.
                y_pred = algorithm.fit_predict(X)
            elif name == "KDE":
                # KDE non plus
                # Score samples
                pred = np.exp(algorithm.fit(X).score_samples(X))
                n = sum(pred < 0.05)
                outlier_ind = np.asarray(pred).argsort()[:n]
                y_pred = np.array(
                    [-1 if i in outlier_ind else 1 for i in range(len(X))]
                )
            elif name == "Tukey":
                # Tukey non plus
                outlier_ind = algorithm.outliers(X)
                y_pred = np.array(
                    [-1 if i in outlier_ind else 1 for i in range(len(X))]
                )
            else:
                y_pred = algorithm.fit(X).predict(X)
                # Affichage des points et des contours de décision
                Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()])
                Z = Z.reshape(xx.shape)
                plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors="black")

            plt.text(
                0.99,
                0.01,
                ("%.2fs" % (t1 - t0)).lstrip("0"),
                transform=plt.gca().transAxes,
                size=15,
                horizontalalignment="right",
            )

            # Ajustement pour identifier en orange les valeurs aberrantes
            y_pred = 1 - y_pred

        colors = np.array(["#377eb8", "#ff7f00"])
        ax.scatter(X[:, 0], X[:, 1], s=10, color=colors[(y_pred + 1) // 2])
        plt.xlim(-7, 7)
        plt.ylim(-7, 7)
        plt.xticks(())
        plt.yticks(())
        plot_num += 1
    plt.show()


In [ ]:
plot_datasets(addOutliers=False)



<!-- #region id="4f3e9a01" -->
Le dernier panneau de la figure montre une absence de distribution cohérente des points. Cette distribution aléatoire
est utilisée afin d'observer le comportement des algorithmes de détection de valeurs aberrantes
en l'absence de distribution cohérente sous-jacente. Les algorithmes font l'hypothèse que les vraies données ont une
organisation corrélée spatialement et que les données aberrantes n'en ont pas.

<!-- #endregion -->



In [ ]:
plot_datasets(addOutliers=True)



<!-- #region id="5e9761fb" -->
Dans la figure ci-dessus et les suivantes, les points identifiés comme valeurs aberrantes (vraies ou fausses)
sont toujours affichés en orange. Les temps de calcul des différentes méthodes sont également
affichés dans le coin inférieur droit de chaque panneau.

Les distributions de données dans les panneaux 1, 2 et 4 devraient être les plus faciles à nettoyer des valeurs aberrantes.

Le panneau 3 est plus difficile puisque les points externes du nuage de points le plus étendu se confondent
avec des valeurs aberrantes.

Le dernier panneau permettra de voir les artefacts introduits par chaque méthode de nettoyage puisqu'il n'y
a pas de différence significative entre les deux distributions de points.
<!-- #endregion -->

<!-- #region id="996b9ca8" -->
## <a id=élimination-des-données-aberrantes>Élimination des données aberrantes</a>
<!-- #endregion -->

<!-- #region id="6613d7c2" -->
À partir d'ici nous allons essayer différentes méthodes de détection de valeurs aberrantes.
Pour chacune des fonctions utilisées, n'hésitez pas à vous référer à la documentation
de [Scikit-learn](https://scikit-learn.org/stable/).

Généralement, une simple recherche sur Google de la fonction (par exemple ci-dessous, cherchez
« *EllipticEnvelope* » sur [Google](https://www.google.com/search?q=EllipticEnvelope)) vous mènera directement
à la bonne page de documentation.

La partie importante de cette documentation, qui n'est pas discutée dans ce module, est l'ensemble des
paramètres arbitrairement fixés à leurs valeurs par défaut que vous voudriez peut-être essayer de corriger
vu que vous connaissez déjà quelles sont les "bonnes" données et les "mauvaises" données.
<!-- #endregion -->

<!-- #region colab_type="text" id="fe901fa8" -->
### <a id=covariance-robuste>Covariance robuste</a>
<!-- #endregion -->



In [ ]:
from sklearn.covariance import EllipticEnvelope

algo = ("Robust covariance", EllipticEnvelope(contamination=outliers_fraction))

plot_datasets(*algo)



<!-- #region id="686917f7" -->
Cette fonction calcule l'ellipse de covariance entre les données. Le pourcentage de contamination de
points aberrants dans l'ellipse peut être ajusté.

L'hypothèse d'une ellipse de covariance basée sur les vraies données n'est pas très bonne ici puisque
les données, sauf dans le premier panneau, sont tout sauf distribuées comme un nuage de point ellipsoïdal.

<!-- #endregion -->

<!-- #region colab_type="text" id="896b210a" -->
### <a id=svm-monoclasse>SVM monoclasse</a>
<!-- #endregion -->



In [ ]:
from sklearn import svm

algo = ("One-Class SVM", svm.OneClassSVM(nu=outliers_fraction, kernel="rbf", gamma=0.4))

plot_datasets(*algo)



<!-- #region id="2c9e41ec" -->
Cette méthode est plus sélective que la précédente, car elle isole les vrais ensembles de données. Elle en ajoute
d'autres malheureusement.
<!-- #endregion -->

<!-- #region colab_type="text" id="6e7c9ed5" -->
### <a id=forêt-disolation>Forêt d'isolation</a>
<!-- #endregion -->



In [ ]:
from sklearn.ensemble import IsolationForest

algo = (
    "Isolation Forest",
    IsolationForest(contamination=outliers_fraction, random_state=seed),
)

plot_datasets(*algo)



<!-- #region id="733fdd09" -->
Cette méthode est encore meilleure et plus discriminante que la précédente.
<!-- #endregion -->

<!-- #region colab_type="text" id="7ece0b7f" -->
### <a id=facteur-daberration-local>Facteur d'aberration local</a>
<!-- #endregion -->



In [ ]:
from sklearn.neighbors import LocalOutlierFactor

algo = (
    "Local Outlier Factor",
    LocalOutlierFactor(n_neighbors=35, contamination=outliers_fraction),
)

plot_datasets(*algo)



<!-- #region id="0a6fb97c" -->
Cette méthode fonctionne plutôt bien; la plupart des valeurs aberrantes dans les quatre premiers panneaux sont identifiées.
<!-- #endregion -->

<!-- #region colab_type="text" id="b610b963" -->
### <a id=test-de-tukey-pour-les-valeurs-extrêmes>Test de Tukey pour les valeurs extrêmes</a>
<!-- #endregion -->



In [ ]:
ecart = 1.5


# Définissons la fonction qui utilise les déviations interquartiles avec les
# quartiles 1 et 3 comme plancher et plafond.
class Tukey:
    def fit(self, X):
        None

    def outliers(self, x):
        q1 = np.percentile(x, 25)
        q3 = np.percentile(x, 75)
        iqr = q3 - q1
        floor = q1 - ecart * iqr
        ceiling = q3 + ecart * iqr
        outlier_indices = np.where((x < floor) | (x > ceiling))[0]
        return outlier_indices


algo = ("Tukey", Tukey())
plot_datasets(*algo)



<!-- #region id="9fc34cf0" -->
Cette méthode statistique fait l'hypothèse que les vraies données sont distribuées selon une gaussienne et
identifie comme aberrantes celles ayant une trop faible probabilité d'appartenir à la gaussienne. Le modèle
marche bien dans le premier panneau. Dans les panneaux, les valeurs des quartiles sont surestimées comme l'est
l'étendue de la gaussienne résultante. Aucun point n'est alors exclu de la gaussienne; il n'y a pas de
valeurs aberrantes détectées.
<!-- #endregion -->

<!-- #region colab_type="text" id="3840c65b" -->
### <a id=estimation-de-densité-par-noyau-kde>Estimation de densité par noyau (KDE)</a>
<!-- #endregion -->



In [ ]:
from sklearn.neighbors import KernelDensity

# Variez le type de kernel ci-dessous pour comparer les résultats
# kernel : [‘gaussian’|’tophat’|’epanechnikov’|’exponential’|’linear’|’cosine’]

algo = ("KDE", KernelDensity(bandwidth=0.2, kernel="gaussian"))

plot_datasets(*algo)



<!-- #region id="4de460d6" -->
Cette méthode marche bien pour les distributions gaussiennes d'une taille donnée. C'est le cas des blobs concentrés
dans les trois premiers panneaux. Le second blob dans le panneau 3 est gaussien, mais trop étendu pour être sélectionné.
<!-- #endregion -->

<!-- #region id="4b2fd1d4" -->
# <a id=conclusion>Conclusion</a>

Les résultats de cette section montrent que l'identification des valeurs aberrantes n'est pas une opération triviale.
Les méthodes de la forêt d'isolation et du facteur d'aberration local performent le mieux avec les jeux de données utilisés ici. Il va donc falloir souvent tester plusieurs méthodes avec vos problèmes pour voir laquelle performe le mieux entre deux ou trois méthodes.
<!-- #endregion -->
